## <p style="text-align: center;">Numerical Algorithms - Homework Assignment 3</p>
**<p style="text-align: center;">VU Numerical Algorithms, summer semester 2018. Due to 17.06.2018. </p>**

### Programming Exercises

#### Effects of Preconditioners on Conjugate Gradient (12 points)

Implement the conjugate gradient method *CG* and experimentally investigate the effect of different preconditioners on its convergence. Please compare standard *CG* and preconditioned *CG* (*PCG*) for three given symmetric positive definite test problems in terms of the number of iterations and in terms of the runtime (including the time for
computing and applying the preconditioner!) until convergence. Show the convergence histories (norm of relative residual vs. iteration number) graphically.

* Implement standard CG efficiently (do not use the CG implementation available in Octave!). In particular, store the sparse matrix in a sparse matrix format
* Use the following preconditioners:
Diagonal preconditioner
  * Block diagonal preconditioner
  * Incomplete Cholesky factorization with no fill-in
  * Incomplete Cholesky factorization with threshold dropping: experiment with different thresholds and discuss the effect of the choice of threshold.
* Test matrices: Please use (at least) the following three test matrices from the
"Matrix Market" (http://math.nist.gov/MatrixMarket/) for your experiments:
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos5.html
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos6.html
  * http://math.nist.gov/MatrixMarket/data/misc/cylshell/s3rmt3m3.html

In [3]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% Execute code for programming assignment.
function execute()
    res = struct();
    
    % Iterate over test matrices.
    filenames = {"nos5.mtx"; "nos6.mtx"; "s3rmt3m3.mtx"};
    for i = 1:length(filenames)
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);

        % Prepare data structure for evaluation metrics.
        res.(filenames{i}).st = struct();
        
        % ----------------------------------------------------
        % 1. Apply standard CG.
        % ----------------------------------------------------
        
        [
            res.(filenames{i}).st.numberOfIterations, ...
            res.(filenames{i}).st.runTime, ...
            res.(filenames{i}).st.relativeResidualHistory
        ] = applyCG(A, 10^-4);
        
        % ----------------------------------------------------
        % 2. Apply CG with preconditioner.
        % ----------------------------------------------------
        
        pcNames = {"diag"; "block"; "cholesky_nofi"; "cholesky_drop"};
        for i = 1:length(pcNames)
            res.(filenames{i}).(pcNames{i}) = struct();
            
            % Configure preconditioner.
            % Note: Parameters used here were determined in prior experiments (see documentation).
            precondConfig = struct();
            precondConfig.method = pcNames{i};
            precondConfig.alpha = 3;
            precondConfig.stepsize = 3;
            precondConfig.dropThreshold = 3;
            
            % Generate matrix for preconditioning.
            M = generatePreconditioning(A, precondConfig);
            
            % Apply CG with preconditioning.
            [
                res.(filenames{i}).(pcNames{i}).numberOfIterations, ...
                res.(filenames{i}).(pcNames{i}).runTime, ...
                res.(filenames{i}).(pcNames{i}).relativeResidualHistory
            ] = applyCG(A, 10^-2, inv(M));
            pcNames{i}
            %res.(filenames{i}).(pcNames{i}).numberOfIterations
            printf("-------")
        end
    end
end

% Execute task.
%execute()

In [ ]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% Investigate effect of threshold for PC with 
% incomplete Cholesky factorization with threshold 
% dropping.
function executeParameterGridSearch()
    starttime = time();
    printf("Starting grid search.\n")
    
    % Define matrix file names.
    filenames = {"nos5.mtx"; "nos6.mtx"; "s3rmt3m3.mtx"};
    
    % Create structure for holding results.
    res = struct();
    
    % ----------------------------------------
    % 1. Grid search for block-diagonale M.
    % ----------------------------------------
    printf("* Grid seach for block-diagonale (block size).\n")
    
    precondConfig = struct();
    precondConfig.method = "block";
    %{
    % Iterate over test matrices.
    for i = 1:length(filenames)
        printf("  - %s\n", filenames{i})
        
        res.(filenames{i}) = struct();
        res.(filenames{i}).numberOfIterations = [];
        res.(filenames{i}).runtime = [];
        res.(filenames{i}).relativeResidualHistory = [];
        
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);        
        
        % Compute CG with block-diag. M with different block sizes.
        stepsizes = 1:round(size(A)(1) / 10):size(A)(1) / 2;
        for stepsize = stepsizes
            precondConfig.stepsize = stepsize;
            M = generatePreconditioning(A, precondConfig);
            [numIter, runtime, relResHistory] = applyCG(A, 10^-8, inv(M));
            res.(filenames{i}).numberOfIterations = [res.(filenames{i}).numberOfIterations numIter];
            res.(filenames{i}).runtime = [res.(filenames{i}).runtime runtime];
        end
    end
    %}
    
    % ----------------------------------------
    % 2. Grid search for incomplete Cholesky 
    % factorization with no fill-in.
    % ----------------------------------------
    printf("* Grid seach for incomplete Cholesky factorization with no fill-in (alpha).\n")
    
    precondConfig = struct();
    precondConfig.method = "cholesky_nofi";
    
    % Iterate over test matrices.
    for i = 1:length(filenames)
        printf("  - %s\n", filenames{i})
    
        res.(filenames{i}) = struct();
        res.(filenames{i}).numberOfIterations = [];
        res.(filenames{i}).runtime = [];
        res.(filenames{i}).relativeResidualHistory = [];
        
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);        
        
        % Compute CG with block-diag. M with different block sizes.
        % For calculation of upperAlphaLimit: https://www.mathworks.com/help/matlab/ref/ichol.html#btuq275-3.
        upperAlphaLimit = max(sum(abs(A), 2) ./ diag(A)) - 2
        alphas = upperAlphaLimit / 10:upperAlphaLimit / 10:upperAlphaLimit;
        for alpha = alphas
            precondConfig.alpha = alpha
            M = generatePreconditioning(A, precondConfig);
            [numIter, runtime, relResHistory] = applyCG(A, 10^-2, inv(M));
            res.(filenames{i}).numberOfIterations = [res.(filenames{i}).numberOfIterations numIter];
            res.(filenames{i}).runtime = [res.(filenames{i}).runtime runtime];
        end

        res.(filenames{i}).runtime
        res.(filenames{i}).numberOfIterations
    end    
    
    % ----------------------------------------
    % 3. Grid search for incomplete Cholesky 
    % factorization with drop threshold for 
    % fill-in.
    % ----------------------------------------
    printf("* Grid seach for incomplete Cholesky factorization with fill-in dropping threshold (alpha, dropping threshold).\n")
   
    printf("Grid search done. Time: %i\n", time() - starttime);
end

% Execute parameter grid search to find decent values for
%    - alpha
%    - stepsize
%    - dropThreshold.
% Note that this search is variant towards the investigated PC-method.
executeParameterGridSearch()

Starting grid search.
* Grid seach for block-diagonale (block size).
* Grid seach for incomplete Cholesky factorization with no fill-in (alpha).
  - nos5.mtx
upperAlphaLimit =  0.84463
ans =

 Columns 1 through 6:

   1.1110e+02   7.5381e-03   3.9182e-03   3.4330e-03   3.4890e-03   3.4099e-03

 Columns 7 through 11:

   4.1850e-03   2.9669e-03   3.4330e-03   3.3741e-03   3.5501e-03

ans =

 Columns 1 through 8:

   149009       14        7        6        6        6        6        5

 Columns 9 through 11:

        6        6        6

  - nos6.mtx
upperAlphaLimit =   -2.5000e-07
